In [ ]:
import requests
from bs4 import BeautifulSoup

def get_registered_players(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # This is a placeholder for the actual logic to find the number of registered players
        # You will need to inspect the HTML structure of each homepage to find the correct element
        # For example, if the number of players is in a <div> with class "players-count":
        players_count = soup.find('div', class_='players-count')
        
        if players_count:
            return players_count.text.strip()
        else:
            return 'Not found'
    except requests.exceptions.RequestException as e:
        return f'Error: {e}'

# Add a new column to the FIFA DataFrame for the number of registered players
FIFA['Registered Players'] = FIFA['Homepage'].apply(get_registered_players)

print(FIFA)

In [ ]:
# the missing countries, run through the google API to find countries.
import requests
import time
import pandas as pd



from dotenv import load_dotenv
import os
load_dotenv()

# Google API Key & Search Engine ID (replace with actual values)
api_key = os.getenv("API_KEY_MO")
SEARCH_ENGINE_ID_MO = '87da01e7d6b6942f9'

# Function to search for the country using Google Custom Search
def get_country_from_place(place_name_MO):
    # Modify query to explicitly ask about the country
    search_query_MO = f"In what country is {place_name_MO} in?"
    search_url_MO = f"https://www.googleapis.com/customsearch/v1?q={search_query_MO}&key={api_key}&cx={SEARCH_ENGINE_ID_MO}"

    
    try:
        response_MO = requests.get(search_url_MO)
        data2_MO = response_MO.json()
        
        # Check if results exist
        if "items" in data2_MO:
            for item in data2_MO["items"]:
                snippet_MO = item.get("snippet", "").upper()  # Convert to uppercase for comparison
                
                # Extract words and check if they match a known country
                words_MO = snippet_MO.split()
                for word_2_MO in words_MO:
                    if word_2_MO in Country_l_MO:  # Check if word is in the existing country list
                        return word_2_MO  # Return matched country
                
        return None  # No valid country found
    except Exception as e:
        print(f"Error for {place_name_MO}: {e}")
        return None

# Iterate through baby_shark and update 'Countries' where needed
for index_MO, row2_MO in baby_shark.iterrows():
    if row2_MO["Country"] == "NO INFORMATION":  # Only search for unknown countries
        if row2_MO["State"] == "UNDISCLOSED":  # Skip if State is 'UNDISCLOSED'
            continue  

        place_name_MO = row2_MO["State"]
        country_MO = get_country_from_place(place_name_MO)
        
        if country_MO:
            baby_shark.at[index_MO, "Country"] = country_MO
        else:
            print(f"Could not find country for: {place_name_MO}")
        
        time.sleep(1)  # Sleep to avoid hitting API rate limits

print("Country lookup completed!")
   
